# 1. 라이브러리 임포트

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

In [ ]:
device = "cpu"

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(device)

# 2. JSON 데이터 로드

## 2-a. JSON 데이터 로딩

In [ ]:
df = pd.read_csv("./KRW-XRP-1m-2024-12-06T07-24-04-827Z.csv")

In [ ]:
print(df.head(2))

## 2-b. 특징 추출

In [ ]:
# 필요한 열 선택
df = df[['candle_date_time_kst', 'opening_price', 'high_price', 'low_price', 'trade_price', 'candle_acc_trade_volume']]

# datetime 형식으로 변환
df['candle_date_time_kst'] = pd.to_datetime(df['candle_date_time_kst'])
df.set_index('candle_date_time_kst', inplace=True)

## 2-c. 결측치, 이상치 처리

In [ ]:
# 결측치 제거
df.dropna(inplace=True)

## 2-d. 데이터 정규화

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df)

# DataFrame으로 변환
scaled_df = pd.DataFrame(scaled_df, index=df.index, columns=df.columns)

In [ ]:
test_df = scaled_df[1900000:]
# test_df = scaled_df
print(test_df)

## 2-e. 시계열 데이터 생성

In [ ]:
def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data) - seq_length):
        x = data.iloc[i:(i+seq_length)].values
        y = data.iloc[i+seq_length]['trade_price']
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

seq_length = 60  # 예: 60분(1시간) 동안의 데이터를 사용하여 다음 값을 예측
X, y = create_sequences(scaled_df, seq_length)
# X, y = create_sequences(test_df, seq_length)

## 2-f. 학습 데이터와 테스트 데이터 분리

In [ ]:
train_size = int(len(X) * 0.8)
X_train = X[:train_size]
y_train = y[:train_size]

X_test = X[train_size:]
y_test = y[train_size:]

## 2-g. 데이터를 PyTorch Tensor로 변환

In [ ]:
# Numpy 배열을 Tensor로 변환
X_train = torch.from_numpy(X_train).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.FloatTensor)

X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.FloatTensor)

# 3. 모델 정의

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, device):
        super(LSTMModel, self).__init__()
        
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM 레이어 정의
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # 출력 레이어 정의
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 초기 hidden state와 cell state 설정
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=self.device).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=self.device).requires_grad_()

        # LSTM 실행
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # 마지막 시점의 출력값 사용
        out = self.fc(out[:, -1, :])
        return out

# 4. 모델 학습

## 4-a. 하이퍼파라미터 설정

In [ ]:
input_size = X_train.shape[2]  # 5
hidden_size = 256
num_layers = 4
output_size = 1
learning_rate = 0.001
num_epochs = 50
batch_size = 1024 # 배치 크기 설정

## 4-b. 모델, 손실 함수, 옵티마이저 정의

In [ ]:
print(device)
model = LSTMModel(input_size, hidden_size, num_layers, output_size,device)
model = LSTMModel(input_size, hidden_size, num_layers, output_size,device).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
print("Available GPUs:", torch.cuda.device_count())

for i in range(torch.cuda.device_count()):
    print(f"GPU {i}:")
    print("  Name:", torch.cuda.get_device_name(i))
    print("  Allocated memory:", torch.cuda.memory_allocated(i) / 1e9, "GB")
    print("  Cached memory:", torch.cuda.memory_reserved(i) / 1e9, "GB")
    print("  Utilization:", torch.cuda.max_memory_allocated(i) / torch.cuda.get_device_properties(i).total_memory * 100, "%")


print("Current GPU in use:", torch.cuda.current_device())

if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
model.to('cuda')

In [ ]:
# Numpy 배열을 Tensor로 변환
X_train = X_train.to(device)
y_train = y_train.to(device)

X_test = X_test.to(device)
y_test = y_test.to(device)

In [ ]:
# 모델 인스턴스가 주어진 경우
device = next(model.parameters()).device
print("Current device:", device)

## 4-c. 학습 루프

In [ ]:
import time
import torch

# 훈련 전 메모리 캐시 초기화 (GPU 사용 시)
if torch.cuda.is_available():
    torch.cuda.empty_cache()

try:
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        start_time = time.time()  # 각 에포크 시작 시간을 기록
        
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train[i:i + batch_size].to(device)
            y_batch = y_train[i:i + batch_size].to(device)

            # 모델 훈련
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch.view(-1, 1))

            optimizer.zero_grad()  # 기울기 초기화
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트
            
            total_loss += loss.item()

        # 에포크가 종료될 때 마다 경과 시간과 평균 손실을 계산 및 출력
        elapsed_time = time.time() - start_time
        avg_loss = total_loss / (len(X_train) / batch_size)
        print(f'Epoch [{epoch}/{num_epochs}], Average Loss: {avg_loss:.4f}, Time: {elapsed_time:.2f} sec')

        # 선택적으로 GPU 메모리 캐시 청소 (필요 시)
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

except Exception as e:
    print(f"Training failed: {e}")

In [ ]:
from datetime import datetime

model_filename = f'model_lr{learning_rate}_bs{batch_size}_epochs{num_epochs}.pth'
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_filename = f'model_lr{learning_rate}_bs{batch_size}_epochs{num_epochs}_{current_time}.pth'

In [ ]:
torch.save(model.state_dict(),f"./model/{model_filename}")

# 5. 모델 평가 및 예측

## 5-a. 테스트 데이터에 대한 예측

In [ ]:
print(len(X_test))

In [ ]:
model.eval()
with torch.no_grad():
    train_predict = model(X_test)

## 5-b. 스케일된 데이터를 원래대로 복원

In [ ]:
# 예측값과 실제값을 스케일러의 inverse_transform을 사용하여 원래 값으로 복원

# 예측값
predicted = train_predict.detach().cpu().numpy()
# 실제값
actual = y_test.detach().cpu().numpy()

# trade_price만 복원하기 위해 다른 컬럼은 0으로 채움
padding = np.zeros((predicted.shape[0], scaled_df.shape[1]-1))
predicted_full = np.concatenate((padding, predicted), axis=1)
actual_full = np.concatenate((padding, actual.reshape(-1,1)), axis=1)

# inverse_transform
predicted_original = scaler.inverse_transform(predicted_full)[:, -1]
actual_original = scaler.inverse_transform(actual_full)[:, -1]

## 5-c. 매수/매도 신호 생성

In [ ]:
# 가격 상승 예측 시 매수, 하락 예측 시 매도
signals = []
for i in range(len(predicted_original)-1):
    if predicted_original[i+1] > actual_original[i]:
        signals.append('Buy')
    else:
        signals.append('Sell')

# 6. 결과 시각화

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.index[-len(predicted_original):], actual_original, label='Actual Price')
plt.plot(df.index[-len(predicted_original):], predicted_original, label='Predicted Price')
plt.legend()
plt.show()

# 7. 매수/매도 신호 표시

In [ ]:
import pandas as pd



signal_df = pd.DataFrame({
    'Date': df.index[-len(signals):],
    'Actual Price': actual_original[:-1],
    'Predicted Price': predicted_original[:-1],
    'Signal': signals
})

pd.set_option('display.max_rows', None)  # 모든 행을 출력하도록 설정
print(signal_df)

print(signal_df.head())

In [ ]:
import pandas as pd

# 초기 자본 및 변수 설정
initial_capital = 1000000  # 100만원
cash = initial_capital  # 초기 현금
coins = 0  # 보유 코인 수
asset_history = []  # 자산 변동 이력

# 거래 로직 실행
for index, row in signal_df.iterrows():
    current_price = row['Predicted Price']  # 현재 가격 사용
    if row['Signal'] == 'Buy' and cash > 0:  # 매수 조건
        coins = cash / current_price  # 모든 현금으로 코인 매수
        cash = 0  # 현금 소모
    elif row['Signal'] == 'Sell' and coins > 0:  # 매도 조건
        cash = coins * current_price  # 모든 코인 매도
        coins = 0  # 코인 소모

    # 현재 자산 계산 (현금 + 코인 가치)
    total_assets = cash + coins * current_price
    asset_history.append(total_assets)  # 자산 이력 기록

# 자산 이력을 DataFrame으로 변환
asset_df = pd.DataFrame(asset_history, columns=['Total Assets'])
asset_df['Date'] = signal_df['Date']  # 날짜 정보 추가

# 결과 출력
print(asset_df)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(asset_df['Date'], asset_df['Total Assets'], label='Total Assets')
plt.xlabel('Date')
plt.ylabel('Assets in KRW')
plt.title('Asset Variation Over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 모델 인스턴스가 주어진 경우
device = next(model.parameters()).device
print("Current device:", device)

In [ ]:
print("hey")

# 모델 저장 경로 설정
model_save_path = "trained_model.pth"

# 모델 저장
torch.save(model.state_dict(), model_save_path)

print(f"Model saved to {model_save_path}")

In [ ]:
print(len(X))